# Q4: Geographic Distribution & Regional Specialization

## Research Questions

This section addresses three related questions:

1. **Geographic distribution:** Where are clinical trials registered, and how concentrated is activity across countries?
2. **Therapeutic specialization:** Are certain countries relatively over- or under-represented in specific therapeutic areas?
3. **Temporal dynamics:** Has the geographic footprint of trials shifted over time?

The analysis is descriptive and registry-based. Results should be interpreted as patterns of **trial registration**, not as measures of disease burden, market size, or execution quality.

---

## What this analysis can support — and where it is limited

The geographic distribution of trials can be characterized with high confidence. Concentration metrics such as the Herfindahl–Hirschman Index (HHI), complemented with bootstrap uncertainty, provide a stable picture of where trials are registered and how concentrated activity is across countries.

Changes in concentration over time can be assessed more cautiously. Divergence measures (e.g. Jensen–Shannon distance) capture shifts in distributional shape, but do not isolate drivers or imply structural change.

By contrast, **country-level therapeutic specialization cannot be established robustly with the available data**. Section 3 therefore presents specialization patterns strictly as exploratory signals rather than firm conclusions. Several structural limitations constrain inference:

- **Condition labels are free-text** and not standardized (e.g. "breast cancer", "breast neoplasm", "carcinoma of breast"), fragmenting denominators.
- **Multi-condition trials** mechanically inflate relative measures such as Location Quotients when counted once per condition.
- **Compositional effects** (phase, sponsor, trial size) are not controlled in the specialization analysis, so apparent differences may reflect portfolio mix rather than geography.
- **Multiplicity** is substantial: hundreds of country–condition pairs are evaluated without formal correction, making isolated signals fragile.

Taken together, these factors mean that specialization results are best viewed as hypothesis-generating.

---

## What this analysis does not measure

This notebook does **not** address:

- **Disease burden or unmet need** — trial counts are not proxies for epidemiology.
- **Market size or commercial opportunity** — more registrations do not imply greater revenue potential.
- **Operational performance** — registration country does not reflect recruitment speed, data quality, or execution risk.

---

## Key assumptions and simplifications

- Each trial is assigned to a single **primary country**, defined as the modal site location (ties broken arbitrarily).
- Approximately 8% of trials are multinational; assigning them to one country likely overstates concentration in large markets (e.g. US, China) and understates participation of satellite countries.
- Trial counts reflect **registrations**, not enrollment capacity or site footprint.

---

## Structure of the analysis

- **§2 Geographic distribution:** Trial counts and concentration (HHI with uncertainty)
- **§3 Therapeutic specialization:** Location Quotients (exploratory)
- **§4 Temporal perspective:** Distributional shifts over cohorts
- **Appendix A:** Site complexity patterns (contextual, descriptive)
- **Appendix B:** Location Quotient heatmap (exploratory visualization)

In [1]:
# ============================================================
# Setup
# ============================================================

import sys
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency, mannwhitneyu, spearmanr
from IPython.display import display, Markdown
import plotly.graph_objects as go

# Project root for imports
PROJECT_ROOT = Path('..')
sys.path.insert(0, str(PROJECT_ROOT))

# Shared utilities
from src.data.loader import load_sql_query, get_db_connection
from src.analysis.viz import DEFAULT_COLORS, create_horizontal_bar_chart
from src.analysis.metrics import calc_cramers_v, interpret_effect_size
from src.analysis.constants import PHASE_ORDER_CLINICAL, COHORT_BINS, COHORT_LABELS

# Paths (validated at setup)
DB_PATH = PROJECT_ROOT / 'data' / 'database' / 'clinical_trials.db'
SQL_PATH = PROJECT_ROOT / 'sql' / 'queries'
assert DB_PATH.exists(), f"DB not found: {DB_PATH}"
assert SQL_PATH.exists(), f"SQL folder not found: {SQL_PATH}"

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
# ============================================================
# Database connection
# ============================================================

conn = get_db_connection(DB_PATH)

---

## 1. Data Loading & Validation

In [3]:
# ============================================================
# 1.1 Load ABT (study-level with geographic features)
# ============================================================

df_abt = load_sql_query('q4_abt.sql', conn, SQL_PATH)

n_studies = len(df_abt)
n_with_location = df_abt['has_location_data'].sum()
pct_location = n_with_location / n_studies * 100
year_min, year_max = df_abt['start_year'].min(), df_abt['start_year'].max()

display(Markdown(f"""
**ABT:** {n_studies:,} studies ({year_min}–{year_max}). Location data: {n_with_location:,} ({pct_location:.0f}%).
"""))

# Filter to studies with location data
df_geo = df_abt[df_abt['has_location_data'] == 1].copy()
n_geo = len(df_geo)


**ABT:** 97,943 studies (1990–2025). Location data: 88,404 (90%).


In [4]:
# ============================================================
# 1.2 Geographic summary statistics
# ============================================================

# Unique countries
n_countries = df_geo['primary_country'].nunique()

# Site complexity distribution
n_single_site = df_geo['is_single_site'].sum()
n_multinational = df_geo['is_multinational'].sum()
n_large_multisite = df_geo['is_large_multisite'].sum()

pct_single = n_single_site / n_geo * 100
pct_multinational = n_multinational / n_geo * 100
pct_large = n_large_multisite / n_geo * 100

# Site count statistics
median_sites = df_geo['n_sites'].median()
mean_sites = df_geo['n_sites'].mean()
max_sites = df_geo['n_sites'].max()
q75_sites = df_geo['n_sites'].quantile(0.75)

display(Markdown(f"""
**Geographic summary (n = {n_geo:,} studies with location data):**

| Metric | Value |
|--------|-------|
| Unique countries | {n_countries:,} |
| Single-site trials | {n_single_site:,} ({pct_single:.1f}%) |
| Multinational trials | {n_multinational:,} ({pct_multinational:.1f}%) |
| Large multi-site (≥10 sites) | {n_large_multisite:,} ({pct_large:.1f}%) |
| Median sites per trial | {median_sites:.0f} |
"""))


**Geographic summary (n = 88,404 studies with location data):**

| Metric | Value |
|--------|-------|
| Unique countries | 175 |
| Single-site trials | 64,557 (73.0%) |
| Multinational trials | 7,196 (8.1%) |
| Large multi-site (≥10 sites) | 8,959 (10.1%) |
| Median sites per trial | 1 |


---

## 2. Geographic Distribution

**Question:** Where are clinical trials conducted?

In [5]:
# ============================================================
# 2.1 Country-level distribution
# ============================================================

# Aggregate by primary country
df_country = (
    df_geo
    .groupby('primary_country')
    .agg(
        n_trials=('study_id', 'nunique'),
        n_interventional=('is_interventional', 'sum'),
        n_industry=('is_industry_sponsor', 'sum'),
        median_sites=('n_sites', 'median'),
        pct_multinational=('is_multinational', 'mean'),
    )
    .reset_index()
    .sort_values('n_trials', ascending=False)
)
df_country['pct_multinational'] = df_country['pct_multinational'] * 100
df_country['pct_interventional'] = df_country['n_interventional'] / df_country['n_trials'] * 100
df_country['pct_industry'] = df_country['n_industry'] / df_country['n_trials'] * 100

top20 = df_country.head(20).copy()

# Concentration metrics
total_trials = df_country['n_trials'].sum()
top5_share = df_country.head(5)['n_trials'].sum() / total_trials * 100
top10_share = df_country.head(10)['n_trials'].sum() / total_trials * 100

# HHI with bootstrap CI
df_country['market_share'] = df_country['n_trials'] / total_trials
hhi = (df_country['market_share'] ** 2).sum() * 10000

np.random.seed(42)
n_bootstrap = 1000
country_array = df_geo['primary_country'].values
hhi_bootstrap = []

for _ in range(n_bootstrap):
    sample_idx = np.random.choice(len(country_array), size=len(country_array), replace=True)
    sample_countries = country_array[sample_idx]
    counts = pd.Series(sample_countries).value_counts()
    shares = counts / counts.sum()
    hhi_boot = (shares ** 2).sum() * 10000
    hhi_bootstrap.append(hhi_boot)

hhi_ci_low = np.percentile(hhi_bootstrap, 2.5)
hhi_ci_high = np.percentile(hhi_bootstrap, 97.5)

# HHI interpretation (DOJ/FTC thresholds)
if hhi < 1500:
    hhi_interpretation = "unconcentrated"
elif hhi < 2500:
    hhi_interpretation = "moderately concentrated"
else:
    hhi_interpretation = "highly concentrated"

# Check if CI spans classification boundaries
ci_spans_1500 = hhi_ci_low < 1500 < hhi_ci_high
ci_spans_2500 = hhi_ci_low < 2500 < hhi_ci_high

if ci_spans_1500 or ci_spans_2500:
    boundary_note = " — CI spans classification boundary, interpret with caution"
else:
    boundary_note = ""

pct_multinational_all = df_geo['is_multinational'].mean() * 100

# Quantify multinational impact on top country
top1_name = df_country.iloc[0]['primary_country']
top1_pct_multinational = df_country.iloc[0]['pct_multinational']
top1_share = df_country.iloc[0]['market_share'] * 100

# How many of top country's trials are multinational?
top1_multinational_trials = df_geo[(df_geo['primary_country'] == top1_name) & (df_geo['is_multinational'] == 1)]
top1_n_multinational = len(top1_multinational_trials)
top1_n_total = df_country.iloc[0]['n_trials']

display(Markdown(f"""
### 2.1 Geographic concentration

**How are trials assigned to countries?**

- **Presence:** A country has ≥1 site in the trial (one trial can have presence in multiple countries)
- **Primary country (used here):** The country with the most sites per trial. For example, a trial with 5 US sites, 2 German sites, and 1 French site is assigned to the US.

**Concentration metrics:**

| Metric | Value | 95% CI |
|--------|-------|--------|
| Top 5 countries | {top5_share:.0f}% of all trials | — |
| Top 10 countries | {top10_share:.0f}% of all trials | — |
| **HHI** | **{hhi:.0f}** ({hhi_interpretation}) | [{hhi_ci_low:.0f}–{hhi_ci_high:.0f}]{boundary_note} |

**What is HHI?** The Herfindahl-Hirschman Index measures market concentration. It ranges from near 0 (highly fragmented) to 10,000 (single-country monopoly). The DOJ/FTC classifies markets as: unconcentrated (<1,500), moderately concentrated (1,500–2,500), or highly concentrated (>2,500).

**Multinational trial impact on {top1_name}:**
- {top1_share:.0f}% of all trials list {top1_name} as their primary country
- Of these, {top1_pct_multinational:.0f}% are multinational trials (assigned to {top1_name} because it has the most sites)
- This inflates {top1_name}'s apparent share. Section 2.1.2 presents a site-weighted alternative that distributes credit across all countries.
"""))

display(Markdown("**Top 20 countries by trial count:**"))
display(
    top20[['primary_country', 'n_trials', 'pct_multinational', 'pct_interventional', 'pct_industry', 'median_sites']]
    .rename(columns={
        'primary_country': 'Country',
        'n_trials': 'Trials',
        'pct_multinational': 'Multinational %',
        'pct_interventional': 'Interventional %',
        'pct_industry': 'Industry %',
        'median_sites': 'Median Sites',
    })
    .style.format({
        'Trials': '{:,.0f}',
        'Multinational %': '{:.0f}%',
        'Interventional %': '{:.1f}%',
        'Industry %': '{:.1f}%',
        'Median Sites': '{:.0f}',
    }).hide(axis='index')
)


### 2.1 Geographic concentration

**How are trials assigned to countries?**

- **Presence:** A country has ≥1 site in the trial (one trial can have presence in multiple countries)
- **Primary country (used here):** The country with the most sites per trial. For example, a trial with 5 US sites, 2 German sites, and 1 French site is assigned to the US.

**Concentration metrics:**

| Metric | Value | 95% CI |
|--------|-------|--------|
| Top 5 countries | 58% of all trials | — |
| Top 10 countries | 73% of all trials | — |
| **HHI** | **1504** (moderately concentrated) | [1484–1525] — CI spans classification boundary, interpret with caution |

**What is HHI?** The Herfindahl-Hirschman Index measures market concentration. It ranges from near 0 (highly fragmented) to 10,000 (single-country monopoly). The DOJ/FTC classifies markets as: unconcentrated (<1,500), moderately concentrated (1,500–2,500), or highly concentrated (>2,500).

**Multinational trial impact on United States:**
- 36% of all trials list United States as their primary country
- Of these, 12% are multinational trials (assigned to United States because it has the most sites)
- This inflates United States's apparent share. Section 2.1.2 presents a site-weighted alternative that distributes credit across all countries.


**Top 20 countries by trial count:**

Country,Trials,Multinational %,Interventional %,Industry %,Median Sites
United States,"31,791",12%,83.8%,29.8%,1
China,"7,780",2%,75.6%,22.9%,1
France,"5,254",6%,60.9%,11.1%,1
Turkey (Türkiye),"3,274",1%,66.2%,2.0%,1
Canada,"3,212",4%,82.3%,12.3%,1
United Kingdom,"2,996",12%,72.0%,27.5%,1
Egypt,"2,689",0%,76.8%,1.0%,1
Germany,"2,664",16%,70.8%,38.1%,1
Italy,"2,504",8%,57.5%,10.9%,1
Spain,"2,186",13%,76.4%,19.9%,1


In [6]:
# ============================================================
# 2.1.1 Sensitivity: Single-country trials only
# ============================================================

df_single_country = df_geo[df_geo['is_multinational'] == 0].copy()
n_single = len(df_single_country)

single_country_counts = df_single_country['primary_country'].value_counts()
single_total = single_country_counts.sum()
single_shares = single_country_counts / single_total
hhi_single = (single_shares ** 2).sum() * 10000
top5_single_share = single_country_counts.head(5).sum() / single_total * 100

if hhi_single < 1500:
    hhi_single_interp = "unconcentrated"
elif hhi_single < 2500:
    hhi_single_interp = "moderately concentrated"
else:
    hhi_single_interp = "highly concentrated"

hhi_delta = hhi - hhi_single

# Bootstrap CI for delta
np.random.seed(43)
single_country_array = df_single_country['primary_country'].values
hhi_delta_bootstrap = []

for _ in range(n_bootstrap):
    sample_all = country_array[np.random.choice(len(country_array), len(country_array), replace=True)]
    shares_all = pd.Series(sample_all).value_counts() / len(sample_all)
    hhi_all_boot = (shares_all ** 2).sum() * 10000
    
    sample_single = single_country_array[np.random.choice(len(single_country_array), len(single_country_array), replace=True)]
    shares_single = pd.Series(sample_single).value_counts() / len(sample_single)
    hhi_single_boot = (shares_single ** 2).sum() * 10000
    
    hhi_delta_bootstrap.append(hhi_all_boot - hhi_single_boot)

hhi_delta_ci_low = np.percentile(hhi_delta_bootstrap, 2.5)
hhi_delta_ci_high = np.percentile(hhi_delta_bootstrap, 97.5)
delta_significant = not (hhi_delta_ci_low <= 0 <= hhi_delta_ci_high)

display(Markdown(f"""
### 2.1.1 Sensitivity: Single-country trials only

| Metric | All | Single-country | Δ [95% CI] |
|--------|-----|----------------|------------|
| N | {n_geo:,} | {n_single:,} | — |
| HHI | {hhi:.0f} | {hhi_single:.0f} | {hhi_delta:+.0f} [{hhi_delta_ci_low:+.0f}, {hhi_delta_ci_high:+.0f}] |

ΔHHI {'excludes' if delta_significant else 'includes'} zero. Classification {'changes' if hhi_interpretation != hhi_single_interp else 'unchanged'} ({hhi_interpretation} → {hhi_single_interp}).
"""))


### 2.1.1 Sensitivity: Single-country trials only

| Metric | All | Single-country | Δ [95% CI] |
|--------|-----|----------------|------------|
| N | 88,404 | 81,208 | — |
| HHI | 1504 | 1423 | +82 [+52, +108] |

ΔHHI excludes zero. Classification changes (moderately concentrated → unconcentrated).


In [7]:
# ============================================================
# 2.1.2 Site-weighted concentration (fractional assignment)
# ============================================================

# Each site contributes 1/n_sites to its country
# Example: A trial with sites in US(2), Germany(1), France(1) assigns:
#   US: 2/4 = 0.5, Germany: 1/4 = 0.25, France: 1/4 = 0.25
query_site_weighted = """
WITH site_contributions AS (
    SELECT 
        l.study_id,
        l.country,
        1.0 / COUNT(*) OVER (PARTITION BY l.study_id) AS fractional_weight
    FROM locations l
    JOIN v_studies_clean s ON l.study_id = s.study_id
    WHERE l.country IS NOT NULL AND l.country != ''
      AND s.is_start_year_in_scope = 1
)
SELECT 
    country,
    SUM(fractional_weight) AS weighted_trials,
    COUNT(DISTINCT study_id) AS n_trials_any_site
FROM site_contributions
GROUP BY country
ORDER BY weighted_trials DESC
"""

df_site_weighted = pd.read_sql(query_site_weighted, conn)

# Calculate site-weighted HHI
total_weighted = df_site_weighted['weighted_trials'].sum()
df_site_weighted['market_share'] = df_site_weighted['weighted_trials'] / total_weighted
hhi_site_weighted = (df_site_weighted['market_share'] ** 2).sum() * 10000

top5_site_share = df_site_weighted.head(5)['weighted_trials'].sum() / total_weighted * 100
top10_site_share = df_site_weighted.head(10)['weighted_trials'].sum() / total_weighted * 100

# Bootstrap CI for site-weighted HHI (optimized: vectorized operations)
np.random.seed(45)
df_sites_boot = pd.read_sql("""
    SELECT l.study_id, l.country
    FROM locations l
    JOIN v_studies_clean s ON l.study_id = s.study_id
    WHERE l.country IS NOT NULL AND l.country != ''
      AND s.is_start_year_in_scope = 1
""", conn)

# Pre-compute study-level weights for efficiency
study_site_counts = df_sites_boot.groupby('study_id').size()
df_sites_boot['weight'] = df_sites_boot['study_id'].map(lambda x: 1.0 / study_site_counts[x])

hhi_site_bootstrap = []
unique_studies = df_sites_boot['study_id'].unique()
n_studies_boot = len(unique_studies)

for _ in range(n_bootstrap):
    # Sample study IDs with replacement
    sampled_idx = np.random.randint(0, n_studies_boot, size=n_studies_boot)
    sampled_studies = unique_studies[sampled_idx]
    
    # Use pandas merge for efficient resampling (faster than concat loop)
    study_counts = pd.Series(sampled_studies).value_counts()
    boot_data = df_sites_boot[df_sites_boot['study_id'].isin(study_counts.index)].copy()
    boot_data['multiplier'] = boot_data['study_id'].map(study_counts)
    boot_data['boot_weight'] = boot_data['weight'] * boot_data['multiplier']
    
    # Aggregate by country
    country_weighted = boot_data.groupby('country')['boot_weight'].sum()
    total_boot = country_weighted.sum()
    if total_boot > 0:
        shares = country_weighted / total_boot
        hhi_boot = (shares ** 2).sum() * 10000
        hhi_site_bootstrap.append(hhi_boot)

hhi_site_ci_low = np.percentile(hhi_site_bootstrap, 2.5)
hhi_site_ci_high = np.percentile(hhi_site_bootstrap, 97.5)

if hhi_site_weighted < 1500:
    hhi_site_interp = "unconcentrated"
elif hhi_site_weighted < 2500:
    hhi_site_interp = "moderately concentrated"
else:
    hhi_site_interp = "highly concentrated"

hhi_delta_method = hhi - hhi_site_weighted
top5_delta = top5_share - top5_site_share
classification_change = hhi_interpretation != hhi_site_interp

display(Markdown(f"""
### 2.1.2 Site-weighted concentration

**Method:** Each site contributes 1/n to its country, where n = total sites in that trial.

*Example: A trial with 4 sites (US: 2, Germany: 1, France: 1) assigns US = 0.5, Germany = 0.25, France = 0.25. This differs from primary-country assignment where the entire trial would count toward US.*

| Metric | Primary Country | Site-Weighted | Δ |
|--------|-----------------|---------------|---|
| Top 5 | {top5_share:.0f}% | {top5_site_share:.0f}% | {top5_delta:+.0f}pp |
| Top 10 | {top10_share:.0f}% | {top10_site_share:.0f}% | {top10_share - top10_site_share:+.0f}pp |
| **HHI** | **{hhi:.0f}** [{hhi_ci_low:.0f}–{hhi_ci_high:.0f}] | **{hhi_site_weighted:.0f}** [{hhi_site_ci_low:.0f}–{hhi_site_ci_high:.0f}] | **{hhi_delta_method:+.0f}** |
| Classification | {hhi_interpretation} | {hhi_site_interp} | {'differs' if classification_change else 'consistent'} |

**Limitation:** Site-weighted HHI counts sites, not capacity. A country with 10 single-patient sites and one with 10 large referral centers contribute equally. This metric measures geographic spread of site locations, not enrollment capacity.
"""))

# ============================================================
# 2.1.3 Enrollment-weighted concentration (sensitivity)
# ============================================================

# Check if enrollment data is available
if 'enrollment' in df_geo.columns:
    df_with_enroll = df_geo[df_geo['enrollment'].notna() & (df_geo['enrollment'] > 0)].copy()
    n_with_enroll = len(df_with_enroll)
    pct_with_enroll = n_with_enroll / n_geo * 100
    
    if n_with_enroll > 1000:  # Only report if sufficient data
        enroll_by_country = df_with_enroll.groupby('primary_country')['enrollment'].sum()
        total_enroll = enroll_by_country.sum()
        enroll_shares = enroll_by_country / total_enroll
        hhi_enroll = (enroll_shares ** 2).sum() * 10000
        
        top5_enroll_share = enroll_shares.nlargest(5).sum() * 100
        
        if hhi_enroll < 1500:
            hhi_enroll_interp = "unconcentrated"
        elif hhi_enroll < 2500:
            hhi_enroll_interp = "moderately concentrated"
        else:
            hhi_enroll_interp = "highly concentrated"
        
        hhi_delta_enroll = hhi - hhi_enroll
        
        display(Markdown(f"""
### 2.1.3 Enrollment-weighted concentration (sensitivity)

Weighting by reported enrollment (n = {n_with_enroll:,}, {pct_with_enroll:.0f}% of trials with data):

| Metric | Trial Count | Enrollment-Weighted | Δ |
|--------|-------------|---------------------|---|
| Top 5 | {top5_share:.0f}% | {top5_enroll_share:.0f}% | {top5_share - top5_enroll_share:+.0f}pp |
| HHI | {hhi:.0f} ({hhi_interpretation}) | {hhi_enroll:.0f} ({hhi_enroll_interp}) | {hhi_delta_enroll:+.0f} |

*Enrollment data has selection bias (larger trials more likely to report). Ranking {'differs' if abs(hhi_delta_enroll) > 200 else 'is similar'} between methods.*
"""))
    else:
        display(Markdown(f"""
### 2.1.3 Enrollment-weighted concentration

*Insufficient enrollment data ({n_with_enroll:,} trials, {pct_with_enroll:.0f}%) for reliable weighting.*
"""))
else:
    display(Markdown("""
### 2.1.3 Enrollment-weighted concentration

*Enrollment data not available in ABT. Trial counts used throughout.*
"""))


### 2.1.2 Site-weighted concentration

**Method:** Each site contributes 1/n to its country, where n = total sites in that trial.

*Example: A trial with 4 sites (US: 2, Germany: 1, France: 1) assigns US = 0.5, Germany = 0.25, France = 0.25. This differs from primary-country assignment where the entire trial would count toward US.*

| Metric | Primary Country | Site-Weighted | Δ |
|--------|-----------------|---------------|---|
| Top 5 | 58% | 57% | +1pp |
| Top 10 | 73% | 72% | +1pp |
| **HHI** | **1504** [1484–1525] | **1388** [1368–1407] | **+116** |
| Classification | moderately concentrated | unconcentrated | differs |

**Limitation:** Site-weighted HHI counts sites, not capacity. A country with 10 single-patient sites and one with 10 large referral centers contribute equally. This metric measures geographic spread of site locations, not enrollment capacity.



### 2.1.3 Enrollment-weighted concentration (sensitivity)

Weighting by reported enrollment (n = 86,131, 97% of trials with data):

| Metric | Trial Count | Enrollment-Weighted | Δ |
|--------|-------------|---------------------|---|
| Top 5 | 58% | 76% | -18pp |
| HHI | 1504 (moderately concentrated) | 2125 (moderately concentrated) | -621 |

*Enrollment data has selection bias (larger trials more likely to report). Ranking differs between methods.*


In [8]:
# ============================================================
# 2.2 Country distribution chart
# ============================================================

# Prepare data for plot (sorted ascending for horizontal bar - largest at top)
plot_data = top20.sort_values('n_trials', ascending=True).tail(15)

# Key values for subtitle
top1 = df_country.iloc[0]
top2 = df_country.iloc[1]

fig_country = create_horizontal_bar_chart(
    data=plot_data,
    value_col='n_trials',
    label_col='primary_country',
    title='Top 15 Countries by Trial Count',
    subtitle=f"{top1['primary_country']}: {top1['n_trials']:,.0f} | {top2['primary_country']}: {top2['n_trials']:,.0f}",
    show_pct=False,
    height=500,
)
fig_country.show()

---

## 3. Regional Specialization (Exploratory)

**Question:** Do countries show relative over-/under-representation in therapeutic areas?

### Limitations

| Issue | Impact |
|-------|--------|
| Free-text conditions | Synonyms counted separately |
| Multi-condition inflation | Trial with 5 conditions → 5× denominator contribution |
| No composition controls | LQ may reflect phase/sponsor mix |
| Multiplicity | ~1,000+ pairs tested without FDR |

**Approach:** Location Quotient (LQ) = (country share in condition) / (global share in condition)

- LQ > 1.5: Over-represented
- LQ ≈ 1: Matches global
- LQ < 0.5: Under-represented

In [9]:
# ============================================================
# 3.1 Data characteristics and inflation quantification
# ============================================================

df_spec = load_sql_query('q4_country_condition.sql', conn, SQL_PATH)

n_combinations = len(df_spec)
n_countries_spec = df_spec['country'].nunique()
n_conditions_spec = df_spec['condition_standardized'].nunique()

# Multi-condition trial statistics
pct_multi_condition = (df_geo['n_conditions'] > 1).mean() * 100
mean_conds_per_trial = df_geo['n_conditions'].mean()
median_conds_per_trial = df_geo['n_conditions'].median()
max_conds_per_trial = df_geo['n_conditions'].max()

# Quantify inflation: effective sample size vs actual
n_actual_trials = n_geo
n_effective_obs = df_geo['n_conditions'].sum()  # Each condition is an observation
inflation_factor = n_effective_obs / n_actual_trials

display(Markdown(f"""
### 3.1 Data characteristics

**Country-condition matrix:** {n_combinations:,} pairs | {n_countries_spec} countries | {n_conditions_spec:,} conditions

**Multi-condition inflation:**

| Metric | Value |
|--------|-------|
| Trials with >1 condition | {pct_multi_condition:.0f}% |
| Mean conditions/trial | {mean_conds_per_trial:.1f} |
| Median conditions/trial | {median_conds_per_trial:.0f} |
| Max conditions/trial | {max_conds_per_trial:,.0f} |
| **Inflation factor** | **{inflation_factor:.1f}×** |

A trial with {mean_conds_per_trial:.0f} conditions contributes {mean_conds_per_trial:.0f} observations to the LQ denominator. This inflates apparent "specialization" for conditions that co-occur with others.

**Multiplicity:** With {n_combinations:,} country-condition pairs tested at LQ > 1.5, we expect ~{int(n_combinations * 0.05):,} false positives at α=0.05 even under the null hypothesis. No FDR adjustment is applied.

*Filters: Countries ≥50 trials, conditions ≥100 global trials, pairs ≥5 trials.*
"""))

# ============================================================
# 3.1.1 Formal association test: χ² with Cramér's V
# ============================================================

# Build contingency table (country × condition)
contingency = df_spec.pivot_table(
    index='country',
    columns='condition_standardized',
    values='n_trials',
    fill_value=0,
    aggfunc='sum'
)

# Chi-square test
chi2, p_chi2, dof, expected = chi2_contingency(contingency)

# Cramér's V effect size
n_obs = contingency.values.sum()
min_dim = min(contingency.shape[0] - 1, contingency.shape[1] - 1)
cramers_v = calc_cramers_v(chi2, n_obs, min_dim)
v_label = interpret_effect_size(cramers_v, metric="v")

# p-value formatting
if p_chi2 < 0.001:
    p_str = "p < 0.001"
else:
    p_str = f"p = {p_chi2:.3f}"

display(Markdown(f"""
### 3.1.1 Country × Condition association (formal test)

| Metric | Value |
|--------|-------|
| χ² | {chi2:,.0f} |
| df | {dof:,} |
| p-value | {p_str} |
| Cramér's V | {cramers_v:.3f} ({v_label}) |
| Observations | {n_obs:,} (inflated by multi-condition) |

**Interpretation:** The χ² test {'' if p_chi2 < 0.05 else 'does not '}reject{'' if p_chi2 < 0.05 else ''} independence ({p_str}), but with {n_obs:,} observations, statistical significance is nearly guaranteed. The effect size (Cramér's V = {cramers_v:.3f}) is **{v_label}**, suggesting {'limited practical' if v_label in ('negligible', 'small') else 'moderate practical' if v_label == 'medium' else 'substantial'} association after accounting for sample size.

*Caveat: Both χ² and V are inflated by multi-condition trials. The association strength reflects data structure, not validated specialization.*
"""))


### 3.1 Data characteristics

**Country-condition matrix:** 2,854 pairs | 71 countries | 183 conditions

**Multi-condition inflation:**

| Metric | Value |
|--------|-------|
| Trials with >1 condition | 36% |
| Mean conditions/trial | 1.8 |
| Median conditions/trial | 1 |
| Max conditions/trial | 343 |
| **Inflation factor** | **1.8×** |

A trial with 2 conditions contributes 2 observations to the LQ denominator. This inflates apparent "specialization" for conditions that co-occur with others.

**Multiplicity:** With 2,854 country-condition pairs tested at LQ > 1.5, we expect ~142 false positives at α=0.05 even under the null hypothesis. No FDR adjustment is applied.

*Filters: Countries ≥50 trials, conditions ≥100 global trials, pairs ≥5 trials.*



### 3.1.1 Country × Condition association (formal test)

| Metric | Value |
|--------|-------|
| χ² | 75,505 |
| df | 12,740 |
| p-value | p < 0.001 |
| Cramér's V | 0.143 (small) |
| Observations | 52,618 (inflated by multi-condition) |

**Interpretation:** The χ² test reject independence (p < 0.001), but with 52,618 observations, statistical significance is nearly guaranteed. The effect size (Cramér's V = 0.143) is **small**, suggesting limited practical association after accounting for sample size.

*Caveat: Both χ² and V are inflated by multi-condition trials. The association strength reflects data structure, not validated specialization.*


In [10]:
# ============================================================
# 3.2 High-LQ patterns
# ============================================================

# Minimum n for reporting
MIN_N_REPORT = 20

df_high_lq = df_spec[(df_spec['location_quotient'] > 1.5) & (df_spec['n_trials'] >= MIN_N_REPORT)].copy()
df_high_lq = df_high_lq.sort_values('location_quotient', ascending=False)

n_high_lq_total = len(df_high_lq)
n_very_high_lq = len(df_high_lq[df_high_lq['location_quotient'] > 2.0])

display(Markdown(f"""
### 3.2 High-LQ summary

{n_high_lq_total:,} pairs have LQ > 1.5 (n ≥ {MIN_N_REPORT}), of which {n_very_high_lq:,} have LQ > 2.0.
"""))

# Show only top 3 countries with top 3 conditions each
major_countries = ['United States', 'China', 'Germany', 'France', 'United Kingdom', 'Japan']

for country in major_countries[:3]:
    country_data = df_high_lq[df_high_lq['country'] == country].head(3)
    if len(country_data) > 0:
        specs = []
        for _, row in country_data.iterrows():
            lq = row['location_quotient']
            n = row['n_trials']
            specs.append(f"- {row['condition_standardized'][:35]} (LQ={lq:.1f}, n={n:,.0f})")
        display(Markdown(f"**{country}:**\n" + "\n".join(specs)))

display(Markdown(f"*{n_high_lq_total - 9:,} additional pairs not shown.*"))

# ============================================================
# 3.3 Sensitivity: Single-condition trials
# ============================================================

display(Markdown("### 3.3 Sensitivity: Single-condition trials"))

query_single_cond_lq = """
WITH 
single_cond_studies AS (
    SELECT c.study_id, LOWER(TRIM(c.condition_name)) AS condition_standardized
    FROM conditions c
    JOIN v_studies_clean s ON c.study_id = s.study_id
    WHERE c.condition_name IS NOT NULL AND TRIM(c.condition_name) != ''
      AND s.is_start_year_in_scope = 1
    GROUP BY c.study_id
    HAVING COUNT(DISTINCT LOWER(TRIM(c.condition_name))) = 1
),
single_study_countries AS (
    SELECT DISTINCT l.study_id, l.country, sc.condition_standardized
    FROM locations l
    JOIN single_cond_studies sc ON l.study_id = sc.study_id
    JOIN v_studies_clean s ON l.study_id = s.study_id
    WHERE l.country IS NOT NULL AND l.country != ''
      AND s.is_start_year_in_scope = 1
),
single_country_condition AS (
    SELECT country, condition_standardized, COUNT(DISTINCT study_id) AS n_trials
    FROM single_study_countries GROUP BY country, condition_standardized
),
single_country_totals AS (
    SELECT country, COUNT(DISTINCT study_id) AS country_total
    FROM single_study_countries GROUP BY country
),
single_condition_totals AS (
    SELECT condition_standardized, COUNT(DISTINCT study_id) AS condition_total
    FROM single_study_countries GROUP BY condition_standardized
),
single_global AS (
    SELECT COUNT(DISTINCT study_id) AS global_total FROM single_study_countries
)
SELECT 
    scc.country, scc.condition_standardized, scc.n_trials,
    ROUND((CAST(scc.n_trials AS REAL) / sct.country_total) / 
          (CAST(scond.condition_total AS REAL) / sg.global_total), 3) AS lq_single
FROM single_country_condition scc
JOIN single_country_totals sct ON scc.country = sct.country
JOIN single_condition_totals scond ON scc.condition_standardized = scond.condition_standardized
CROSS JOIN single_global sg
WHERE sct.country_total >= 30 AND scond.condition_total >= 50 AND scc.n_trials >= 10
"""
df_single_lq = pd.read_sql(query_single_cond_lq, conn)

pct_single = 100 - pct_multi_condition

# Merge to compare full vs single-condition LQ
df_lq_compare = df_spec.merge(
    df_single_lq[['country', 'condition_standardized', 'lq_single']],
    on=['country', 'condition_standardized'],
    how='inner'
)
df_lq_compare['lq_delta'] = df_lq_compare['lq_single'] - df_lq_compare['location_quotient']
df_lq_compare['is_unstable'] = df_lq_compare['lq_delta'].abs() > 0.3

# Calculate overall stability statistics
n_comparable = len(df_lq_compare)
n_unstable = df_lq_compare['is_unstable'].sum()
pct_unstable = n_unstable / n_comparable * 100 if n_comparable > 0 else 0

# Among high-LQ pairs specifically
df_high_lq_compare = df_lq_compare[df_lq_compare['location_quotient'] > 1.5]
n_high_comparable = len(df_high_lq_compare)
n_high_unstable = df_high_lq_compare['is_unstable'].sum() if n_high_comparable > 0 else 0
pct_high_unstable = n_high_unstable / n_high_comparable * 100 if n_high_comparable > 0 else 0

display(Markdown(f"""
**Stability assessment:**

| Subset | N pairs | Unstable (|Δ| > 0.3) | % Unstable |
|--------|---------|----------------------|------------|
| All comparable | {n_comparable:,} | {n_unstable:,} | {pct_unstable:.0f}% |
| High-LQ (>1.5) | {n_high_comparable:,} | {n_high_unstable:,} | {pct_high_unstable:.0f}% |

*"Unstable" means the LQ changes by more than 0.3 when restricting to single-condition trials. High instability suggests multi-condition inflation is driving the apparent specialization.*
"""))

# Show specific examples for major countries
sensitivity_rows = []
for country in major_countries[:5]:
    country_top = df_high_lq[df_high_lq['country'] == country].head(1)
    if len(country_top) == 0:
        continue
    
    top_cond = country_top.iloc[0]['condition_standardized']
    full_lq = country_top.iloc[0]['location_quotient']
    
    row = {
        'Country': country, 
        'Condition': top_cond[:20] + '...' if len(top_cond) > 20 else top_cond, 
        'All': f"{full_lq:.2f}"
    }
    
    single_match = df_single_lq[(df_single_lq['country'] == country) & (df_single_lq['condition_standardized'] == top_cond)]
    if len(single_match) > 0:
        single_lq = single_match.iloc[0]['lq_single']
        delta = single_lq - full_lq
        row['Single-cond'] = f"{single_lq:.2f}"
        row['Δ'] = f"{delta:+.2f}"
        row['Stable?'] = 'No' if abs(delta) > 0.3 else 'Yes'
    else:
        row['Single-cond'] = '—'
        row['Δ'] = '—'
        row['Stable?'] = '—'
    
    sensitivity_rows.append(row)

if sensitivity_rows:
    display(Markdown("**Top condition per country (stability check):**"))
    display(pd.DataFrame(sensitivity_rows).style.hide(axis='index'))


### 3.2 High-LQ summary

215 pairs have LQ > 1.5 (n ≥ 20), of which 127 have LQ > 2.0.


**United States:**
- unspecified adult solid tumor, prot (LQ=2.3, n=112)
- leukemia (LQ=2.0, n=218)
- myelodysplastic syndrome (LQ=2.0, n=75)

**China:**
- advanced solid tumor (LQ=5.1, n=63)
- acute ischemic stroke (LQ=4.4, n=43)
- hepatocellular carcinoma (LQ=4.1, n=113)

**Germany:**
- carcinoma, non-small-cell lung (LQ=3.6, n=24)
- pulmonary disease, chronic obstruct (LQ=3.4, n=30)
- crohn's disease (LQ=3.0, n=22)

*206 additional pairs not shown.*

### 3.3 Sensitivity: Single-condition trials


**Stability assessment:**

| Subset | N pairs | Unstable (|Δ| > 0.3) | % Unstable |
|--------|---------|----------------------|------------|
| All comparable | 714 | 318 | 45% |
| High-LQ (>1.5) | 317 | 220 | 69% |

*"Unstable" means the LQ changes by more than 0.3 when restricting to single-condition trials. High instability suggests multi-condition inflation is driving the apparent specialization.*


**Top condition per country (stability check):**

Country,Condition,All,Single-cond,Δ,Stable?
United States,unspecified adult so...,2.34,2.49,+0.15,Yes
China,advanced solid tumor,5.13,5.80,+0.66,No
Germany,"carcinoma, non-small...",3.60,3.84,+0.24,Yes
France,crohn disease,2.38,3.07,+0.69,No
United Kingdom,"pulmonary disease, c...",2.89,4.12,+1.24,No


In [11]:
# ============================================================
# 3.4 Section conclusion
# ============================================================

display(Markdown(f"""
### 3.4 Section conclusion

**Question:** Are there regional specializations in therapeutic areas?

**Answer:** This analysis cannot reliably answer this question due to data limitations:

1. **Label fragmentation:** {n_conditions_spec:,} unique condition strings, many of which are synonyms (e.g., "breast cancer" vs "breast neoplasm")
2. **Multi-condition inflation:** {inflation_factor:.1f}× inflation from trials listing multiple conditions
3. **Statistical noise:** {n_combinations:,} country-condition pairs tested without FDR correction
4. **Unstable estimates:** {pct_unstable:.0f}% of LQ values change substantially when restricting to single-condition trials

The χ² test (§3.1.1) shows a statistically significant association between country and condition (expected with N = {n_obs:,}), but the effect size (Cramér's V = {cramers_v:.3f}, {v_label}) suggests limited practical significance.

**Recommendation:** Do not use LQ patterns from this section for strategic decisions. If regional specialization is a priority question, invest in condition standardization (e.g., MeSH mapping) before re-analysis.

*See Appendix B for an exploratory LQ heatmap.*
"""))


### 3.4 Section conclusion

**Question:** Are there regional specializations in therapeutic areas?

**Answer:** This analysis cannot reliably answer this question due to data limitations:

1. **Label fragmentation:** 183 unique condition strings, many of which are synonyms (e.g., "breast cancer" vs "breast neoplasm")
2. **Multi-condition inflation:** 1.8× inflation from trials listing multiple conditions
3. **Statistical noise:** 2,854 country-condition pairs tested without FDR correction
4. **Unstable estimates:** 45% of LQ values change substantially when restricting to single-condition trials

The χ² test (§3.1.1) shows a statistically significant association between country and condition (expected with N = 52,618), but the effect size (Cramér's V = 0.143, small) suggests limited practical significance.

**Recommendation:** Do not use LQ patterns from this section for strategic decisions. If regional specialization is a priority question, invest in condition standardization (e.g., MeSH mapping) before re-analysis.

*See Appendix B for an exploratory LQ heatmap.*


---

## 4. Temporal Trends in Geographic Distribution

**Question:** Has the geographic distribution of trials shifted over time?

In [12]:
# ============================================================
# 4.1 Temporal trends
# ============================================================

from scipy.spatial.distance import jensenshannon

df_geo['start_cohort'] = pd.cut(df_geo['start_year'], bins=COHORT_BINS, labels=COHORT_LABELS)

n_cohorts = len(COHORT_LABELS)

display(Markdown(f"""
### 4.1 Temporal analysis

**Cohorts:** {', '.join(COHORT_LABELS)}

*Registry coverage changed over time (FDAAA 2007 mandated US registration; non-US registries expanded after 2010).*
"""))

# ============================================================
# 4.1.0 Registry coverage quantification pre/post 2007
# ============================================================

# Define pre/post FDAAA periods
df_geo['era'] = df_geo['start_year'].apply(lambda y: 'Pre-2007' if y < 2007 else 'Post-2007')

# Regional groupings (simplified)
def assign_region(country):
    us_canada = ['United States', 'Canada']
    europe = ['Germany', 'France', 'United Kingdom', 'Italy', 'Spain', 'Netherlands', 
              'Belgium', 'Sweden', 'Austria', 'Switzerland', 'Poland', 'Denmark',
              'Norway', 'Finland', 'Ireland', 'Portugal', 'Greece', 'Czech Republic']
    asia = ['China', 'Japan', 'South Korea', 'India', 'Taiwan', 'Thailand', 
            'Singapore', 'Malaysia', 'Hong Kong', 'Indonesia', 'Philippines']
    
    if country in us_canada:
        return 'North America'
    elif country in europe:
        return 'Europe'
    elif country in asia:
        return 'Asia-Pacific'
    else:
        return 'Other'

df_geo['region'] = df_geo['primary_country'].apply(assign_region)

# Calculate coverage by era and region
era_region = df_geo.groupby(['era', 'region']).agg(
    n_trials=('study_id', 'nunique')
).reset_index()

era_totals = df_geo.groupby('era').agg(n_total=('study_id', 'nunique')).reset_index()
era_region = era_region.merge(era_totals, on='era')
era_region['share_pct'] = era_region['n_trials'] / era_region['n_total'] * 100

# Pivot for display
era_pivot = era_region.pivot_table(
    index='region', 
    columns='era', 
    values='share_pct',
    fill_value=0
).round(1)

# Reorder
region_order = ['North America', 'Europe', 'Asia-Pacific', 'Other']
era_pivot = era_pivot.reindex([r for r in region_order if r in era_pivot.index])

# Calculate absolute counts for context
era_counts = era_region.pivot_table(
    index='region',
    columns='era',
    values='n_trials',
    fill_value=0
)
era_counts = era_counts.reindex([r for r in region_order if r in era_counts.index])

# Pre-2007 total
n_pre_2007 = df_geo[df_geo['era'] == 'Pre-2007']['study_id'].nunique()
n_post_2007 = df_geo[df_geo['era'] == 'Post-2007']['study_id'].nunique()
ratio_post_pre = n_post_2007 / n_pre_2007 if n_pre_2007 > 0 else float('inf')

# Key shifts
us_pre = era_pivot.loc['North America', 'Pre-2007'] if 'North America' in era_pivot.index else 0
us_post = era_pivot.loc['North America', 'Post-2007'] if 'North America' in era_pivot.index else 0
asia_pre = era_pivot.loc['Asia-Pacific', 'Pre-2007'] if 'Asia-Pacific' in era_pivot.index else 0
asia_post = era_pivot.loc['Asia-Pacific', 'Post-2007'] if 'Asia-Pacific' in era_pivot.index else 0

display(Markdown(f"""
### 4.1.0 Registry coverage: Pre vs Post FDAAA (2007)

| Era | N Trials |
|-----|----------|
| Pre-2007 | {n_pre_2007:,} |
| Post-2007 | {n_post_2007:,} |
| **Ratio** | **{ratio_post_pre:.1f}×** |

**Regional share by era (%):**
"""))

display(era_pivot.style.format("{:.1f}%"))

display(Markdown(f"""
**Key shifts:**
- North America: {us_pre:.0f}% → {us_post:.0f}% ({us_post - us_pre:+.0f}pp)
- Asia-Pacific: {asia_pre:.0f}% → {asia_post:.0f}% ({asia_post - asia_pre:+.0f}pp)

*Pre-2007 data is predominantly US-centric due to ClinicalTrials.gov mandate. Non-US registries (EUCTR, ChiCTR, CTRI) expanded after 2007. Temporal trends in §4.1.1–4.1.3 reflect both real shifts and changing coverage.*
"""))

# ============================================================
# 4.1.1 Jensen-Shannon Divergence with bootstrap CI
# ============================================================

all_countries = df_geo['primary_country'].unique()
n_categories = len(all_countries)

cohort_distributions = {}
cohort_ns = {}
for cohort in COHORT_LABELS:
    cohort_data = df_geo[df_geo['start_cohort'] == cohort]
    cohort_ns[cohort] = len(cohort_data)
    if len(cohort_data) > 0:
        counts = cohort_data['primary_country'].value_counts()
        dist = pd.Series(0.0, index=all_countries)
        dist.update(counts / counts.sum())
        cohort_distributions[cohort] = dist.values

jsd_results = []
cohort_list = list(cohort_distributions.keys())
for i in range(len(cohort_list) - 1):
    c1, c2 = cohort_list[i], cohort_list[i + 1]
    jsd = jensenshannon(cohort_distributions[c1], cohort_distributions[c2])
    jsd_results.append({'Comparison': f'{c1} → {c2}', 'JSD': jsd, 'n1': cohort_ns[c1], 'n2': cohort_ns[c2]})

jsd_first_last = None
if len(cohort_list) >= 2:
    jsd_first_last = jensenshannon(cohort_distributions[cohort_list[0]], cohort_distributions[cohort_list[-1]])
    jsd_results.append({
        'Comparison': f'{cohort_list[0]} → {cohort_list[-1]}', 
        'JSD': jsd_first_last,
        'n1': cohort_ns[cohort_list[0]],
        'n2': cohort_ns[cohort_list[-1]]
    })

df_jsd = pd.DataFrame(jsd_results)
max_jsd = df_jsd['JSD'].max()

if max_jsd < 0.05:
    jsd_interp = "negligible"
elif max_jsd < 0.10:
    jsd_interp = "minor"
elif max_jsd < 0.20:
    jsd_interp = "moderate"
else:
    jsd_interp = "substantial"

# Bootstrap CI for JSD (first to last)
np.random.seed(44)
n_jsd_bootstrap = 500
first_cohort_data = df_geo[df_geo['start_cohort'] == cohort_list[0]]['primary_country'].values
last_cohort_data = df_geo[df_geo['start_cohort'] == cohort_list[-1]]['primary_country'].values

jsd_bootstrap = []
for _ in range(n_jsd_bootstrap):
    boot_first = np.random.choice(first_cohort_data, size=len(first_cohort_data), replace=True)
    boot_last = np.random.choice(last_cohort_data, size=len(last_cohort_data), replace=True)
    all_countries_boot = np.union1d(np.unique(boot_first), np.unique(boot_last))
    dist_first = pd.Series(boot_first).value_counts().reindex(all_countries_boot, fill_value=0) / len(boot_first)
    dist_last = pd.Series(boot_last).value_counts().reindex(all_countries_boot, fill_value=0) / len(boot_last)
    jsd_boot = jensenshannon(dist_first.values, dist_last.values)
    jsd_bootstrap.append(jsd_boot)

jsd_ci_low = np.percentile(jsd_bootstrap, 2.5)
jsd_ci_high = np.percentile(jsd_bootstrap, 97.5)

display(Markdown("### 4.1.1 Distributional shift (JSD)"))
display(df_jsd[['Comparison', 'JSD', 'n1', 'n2']].style.format({'JSD': '{:.3f}', 'n1': '{:,}', 'n2': '{:,}'}).hide(axis='index'))

if jsd_first_last is not None:
    jsd_excludes_zero = jsd_ci_low > 0
    display(Markdown(f"""
**{cohort_list[0]} → {cohort_list[-1]}:** JSD = {jsd_first_last:.3f} [95% CI: {jsd_ci_low:.3f}–{jsd_ci_high:.3f}] — {jsd_interp} shift

*JSD measures distributional difference (0 = identical, 1 = completely different). The 1990s cohort has limited coverage ({cohort_ns[cohort_list[0]]:,} trials), so comparisons involving it may reflect registry expansion rather than real geographic shifts.*
"""))

# ============================================================
# 4.1.2 Share by cohort
# ============================================================

top10_countries = df_country.head(10)['primary_country'].tolist()
cohort_global_totals = df_geo.groupby('start_cohort', observed=True).size().reset_index(name='global_total')

temporal_global = (
    df_geo[df_geo['primary_country'].isin(top10_countries)]
    .groupby(['start_cohort', 'primary_country'], observed=True)
    .size()
    .reset_index(name='n_trials')
)
temporal_global = temporal_global.merge(cohort_global_totals, on='start_cohort')
temporal_global['global_share'] = temporal_global['n_trials'] / temporal_global['global_total'] * 100

temporal_global_pivot = temporal_global.pivot_table(
    index='primary_country', columns='start_cohort', values='global_share', fill_value=0, observed=True
).round(1).reindex(top10_countries)

display(Markdown("### 4.1.2 Share by country and cohort (%)"))
display(temporal_global_pivot.style.format("{:.1f}%"))

# Top-10 combined with CI
top10_combined = temporal_global.groupby('start_cohort', observed=True)['n_trials'].sum().reset_index()
top10_combined = top10_combined.merge(cohort_global_totals, on='start_cohort')
top10_combined['top10_share'] = top10_combined['n_trials'] / top10_combined['global_total'] * 100

direction = "unknown"
delta_top10 = 0
delta_ci_low = 0
delta_ci_high = 0

if len(top10_combined) >= 2:
    delta_top10 = top10_combined.iloc[-1]['top10_share'] - top10_combined.iloc[0]['top10_share']
    direction = "diversifying" if delta_top10 < -3 else "concentrating" if delta_top10 > 3 else "stable"
    
    # Bootstrap CI for delta
    np.random.seed(46)
    first_cohort_countries = df_geo[df_geo['start_cohort'] == cohort_list[0]]['primary_country']
    last_cohort_countries = df_geo[df_geo['start_cohort'] == cohort_list[-1]]['primary_country']
    
    delta_bootstrap = []
    for _ in range(500):
        boot_first = np.random.choice(first_cohort_countries.values, size=len(first_cohort_countries), replace=True)
        boot_last = np.random.choice(last_cohort_countries.values, size=len(last_cohort_countries), replace=True)
        
        first_top10 = sum(c in top10_countries for c in boot_first) / len(boot_first) * 100
        last_top10 = sum(c in top10_countries for c in boot_last) / len(boot_last) * 100
        delta_bootstrap.append(last_top10 - first_top10)
    
    delta_ci_low = np.percentile(delta_bootstrap, 2.5)
    delta_ci_high = np.percentile(delta_bootstrap, 97.5)
    
    display(Markdown(f"""
**Top-10 share:** {top10_combined.iloc[0]['top10_share']:.0f}% → {top10_combined.iloc[-1]['top10_share']:.0f}% (Δ = {delta_top10:+.0f}pp [95% CI: {delta_ci_low:+.0f}–{delta_ci_high:+.0f}pp]) — {direction}
"""))

# ============================================================
# 4.1.3 Key country shifts
# ============================================================

top5_countries = df_country.head(5)['primary_country'].tolist()
earliest_cohort = cohort_list[1] if len(cohort_list) > 1 else cohort_list[0]
latest_cohort = cohort_list[-1]

if earliest_cohort in temporal_global_pivot.columns and latest_cohort in temporal_global_pivot.columns:
    display(Markdown(f"### 4.1.3 Share changes ({earliest_cohort} → {latest_cohort})"))
    
    shifts = []
    for country in top5_countries:
        if country in temporal_global_pivot.index:
            early = temporal_global_pivot.loc[country, earliest_cohort]
            late = temporal_global_pivot.loc[country, latest_cohort]
            delta = late - early
            direction_arrow = '↑' if delta > 5 else '↓' if delta < -5 else '→'
            shifts.append({'Country': country, 'Early': f"{early:.0f}%", 'Late': f"{late:.0f}%", 'Δ': f"{delta:+.0f}pp", '': direction_arrow})
    
    if shifts:
        display(pd.DataFrame(shifts).style.hide(axis='index'))


### 4.1 Temporal analysis

**Cohorts:** 1990-1999, 2000-2009, 2010-2019, 2020-2025

*Registry coverage changed over time (FDAAA 2007 mandated US registration; non-US registries expanded after 2010).*



### 4.1.0 Registry coverage: Pre vs Post FDAAA (2007)

| Era | N Trials |
|-----|----------|
| Pre-2007 | 7,554 |
| Post-2007 | 80,850 |
| **Ratio** | **10.7×** |

**Regional share by era (%):**


era,Post-2007,Pre-2007
region,,
North America,37.2%,64.9%
Europe,27.8%,22.2%
Asia-Pacific,17.6%,6.4%
Other,17.3%,6.6%



**Key shifts:**
- North America: 65% → 37% (-28pp)
- Asia-Pacific: 6% → 18% (+11pp)

*Pre-2007 data is predominantly US-centric due to ClinicalTrials.gov mandate. Non-US registries (EUCTR, ChiCTR, CTRI) expanded after 2007. Temporal trends in §4.1.1–4.1.3 reflect both real shifts and changing coverage.*


### 4.1.1 Distributional shift (JSD)

...

### 4.1.2 Share by country and cohort (%)

start_cohort,1990-1999,2000-2009,2010-2019,2020-2025
primary_country,,,,
United States,75.7%,53.2%,36.8%,27.4%
China,0.7%,1.7%,6.8%,13.8%
France,2.5%,4.4%,6.4%,6.1%
Turkey (Türkiye),0.0%,0.4%,1.8%,7.1%
Canada,2.2%,4.7%,4.0%,2.9%
United Kingdom,3.5%,3.1%,3.9%,2.9%
Egypt,0.0%,0.1%,2.1%,5.2%
Germany,1.9%,4.5%,3.6%,1.9%
Italy,2.1%,2.1%,2.6%,3.4%



**Top-10 share:** 89% → 74% (Δ = -15pp [95% CI: -17–-13pp]) — diversifying


### 4.1.3 Share changes (2000-2009 → 2020-2025)

Country,Early,Late,Δ,
United States,53%,27%,-26pp,↓
China,2%,14%,+12pp,↑
France,4%,6%,+2pp,→
Turkey (Türkiye),0%,7%,+7pp,↑
Canada,5%,3%,-2pp,→


---

## 5. Summary & Implications

In [13]:
# ============================================================
# 5. Summary
# ============================================================

top_country_name = df_country.iloc[0]['primary_country']
top_country_share = df_country.iloc[0]['market_share'] * 100
n_high_lq = len(df_high_lq)

display(Markdown(f"""
## Summary

| Question | Finding | Confidence |
|----------|---------|------------|
| Where are trials registered? | {top_country_name} leads ({top_country_share:.0f}%); HHI = {hhi_site_weighted:.0f} ({hhi_site_interp}) | **High** |
| Are there regional specializations? | Cannot answer — data quality insufficient | Low |
| Is concentration changing? | JSD = {max_jsd:.3f} ({jsd_interp} shift); {direction} | Medium |

---

### Distribution (§2)

The clinical trial market is **{hhi_site_interp}** based on site-weighted HHI = {hhi_site_weighted:.0f} [95% CI: {hhi_site_ci_low:.0f}–{hhi_site_ci_high:.0f}].

- Primary-country HHI ({hhi:.0f}) exceeds site-weighted by {hhi_delta_method:.0f} points because multinational trials are assigned entirely to their lead country
- The top 10 countries account for {top10_share:.0f}% of trials (primary-country method) or {top10_site_share:.0f}% (site-weighted)

### Specialization (§3)

The analysis identified {n_high_lq:,} country-condition pairs with LQ > 1.5, but these findings are unreliable due to:
- Free-text condition labels causing synonyms to be counted separately
- Multi-condition trials inflating LQ denominators by {inflation_factor:.1f}×
- No multiple-testing correction across {n_combinations:,} comparisons

The χ² test rejects independence (p < 0.001), but Cramér's V = {cramers_v:.3f} ({v_label}) indicates limited practical association strength.

### Temporal (§4)

The geographic distribution has shifted over time:
- JSD = {jsd_first_last:.3f} [95% CI: {jsd_ci_low:.3f}–{jsd_ci_high:.3f}] comparing {cohort_list[0]} to {cohort_list[-1]}
- Top-10 country share: {direction} (Δ = {delta_top10:+.0f}pp)

However, pre-2007 data is heavily US-centric due to ClinicalTrials.gov being the dominant registry before FDAAA. Post-2007 expansion of non-US registries (EUCTR, ChiCTR, CTRI) means observed geographic diversification may partly reflect improved coverage rather than real shifts in trial activity.

---

**Bottom line:** This analysis reliably answers *where trials are registered* (high confidence). It cannot reliably answer *whether countries specialize in therapeutic areas* (low confidence). Temporal patterns should be interpreted with caution due to registry coverage changes.
"""))


## Summary

| Question | Finding | Confidence |
|----------|---------|------------|
| Where are trials registered? | United States leads (36%); HHI = 1388 (unconcentrated) | **High** |
| Are there regional specializations? | Cannot answer — data quality insufficient | Low |
| Is concentration changing? | JSD = 0.433 (substantial shift); diversifying | Medium |

---

### Distribution (§2)

The clinical trial market is **unconcentrated** based on site-weighted HHI = 1388 [95% CI: 1368–1407].

- Primary-country HHI (1504) exceeds site-weighted by 116 points because multinational trials are assigned entirely to their lead country
- The top 10 countries account for 73% of trials (primary-country method) or 72% (site-weighted)

### Specialization (§3)

The analysis identified 215 country-condition pairs with LQ > 1.5, but these findings are unreliable due to:
- Free-text condition labels causing synonyms to be counted separately
- Multi-condition trials inflating LQ denominators by 1.8×
- No multiple-testing correction across 2,854 comparisons

The χ² test rejects independence (p < 0.001), but Cramér's V = 0.143 (small) indicates limited practical association strength.

### Temporal (§4)

The geographic distribution has shifted over time:
- JSD = 0.433 [95% CI: 0.427–0.455] comparing 1990-1999 to 2020-2025
- Top-10 country share: diversifying (Δ = -15pp)

However, pre-2007 data is heavily US-centric due to ClinicalTrials.gov being the dominant registry before FDAAA. Post-2007 expansion of non-US registries (EUCTR, ChiCTR, CTRI) means observed geographic diversification may partly reflect improved coverage rather than real shifts in trial activity.

---

**Bottom line:** This analysis reliably answers *where trials are registered* (high confidence). It cannot reliably answer *whether countries specialize in therapeutic areas* (low confidence). Temporal patterns should be interpreted with caution due to registry coverage changes.


---

## Appendix A: Site Complexity by Phase/Sponsor

*These patterns inform the multinational trial composition discussed in §2.1. Phase 3 trials are more likely to be multinational, which affects primary-country assignment.*

In [14]:
# ============================================================
# A.1 Site complexity by phase
# ============================================================

df_phase_sites = df_geo[
    (df_geo['is_interventional'] == 1) &
    (df_geo['phase_group'].notna()) &
    (df_geo['phase_group'] != 'Not Applicable') &
    (df_geo['phase_group'] != 'Other')
].copy()

df_phase_sites['phase_group'] = pd.Categorical(df_phase_sites['phase_group'], categories=PHASE_ORDER_CLINICAL, ordered=True)

phase_summary = (
    df_phase_sites
    .groupby('phase_group', observed=True)
    .agg(
        n_trials=('study_id', 'nunique'),
        median_sites=('n_sites', 'median'),
        mean_sites=('n_sites', 'mean'),
        q75_sites=('n_sites', lambda x: x.quantile(0.75)),
        pct_multinational=('is_multinational', 'mean'),
    )
    .reset_index()
)
phase_summary['pct_multinational'] = phase_summary['pct_multinational'] * 100

display(Markdown("### A.1 Site complexity by phase (interventional)"))
display(
    phase_summary
    .rename(columns={
        'phase_group': 'Phase', 'n_trials': 'N', 'median_sites': 'Median',
        'mean_sites': 'Mean', 'q75_sites': 'Q75', 'pct_multinational': 'Multinational %',
    })
    .style.format({'N': '{:,.0f}', 'Median': '{:.0f}', 'Mean': '{:.1f}', 'Q75': '{:.0f}', 'Multinational %': '{:.1f}%'})
    .hide(axis='index')
)

# Key values for reference
p1_sites = phase_summary.loc[phase_summary['phase_group'] == 'Phase 1', 'median_sites'].values
p3_sites = phase_summary.loc[phase_summary['phase_group'] == 'Phase 3', 'median_sites'].values
p1_val = int(p1_sites[0]) if len(p1_sites) > 0 else 'N/A'
p3_val = int(p3_sites[0]) if len(p3_sites) > 0 else 'N/A'
site_ratio = p3_val / p1_val if isinstance(p1_val, int) and p1_val > 0 else 'N/A'

display(Markdown(f"Phase 3 median = {p3_val} sites vs Phase 1 = {p1_val} ({site_ratio:.0f}× ratio)"))

### A.1 Site complexity by phase (interventional)

Phase,N,Median,Mean,Q75,Multinational %
Early Phase 1,890,1,1.4,1,0.6%
Phase 1,"7,482",1,2.8,2,8.9%
Phase 1/2,"2,614",1,5.5,5,14.3%
Phase 2,"10,159",1,11.1,8,16.8%
Phase 2/3,"1,118",1,9.6,3,13.5%
Phase 3,"6,333",4,37.2,37,31.6%
Phase 4,"5,277",1,5.4,2,6.4%


Phase 3 median = 4 sites vs Phase 1 = 1 (4× ratio)

In [15]:
# ============================================================
# A.2 Sponsor effect on site count (Phase 3)
# ============================================================

p3_data = df_phase_sites[df_phase_sites['phase_group'] == 'Phase 3'].copy()
p3_industry = p3_data[p3_data['is_industry_sponsor'] == 1]['n_sites']
p3_non_industry = p3_data[p3_data['is_industry_sponsor'] == 0]['n_sites']

u_stat, p_mw = mannwhitneyu(p3_industry, p3_non_industry, alternative='two-sided')
n1, n2 = len(p3_industry), len(p3_non_industry)
r_biserial = 1 - (2 * u_stat) / (n1 * n2)
effect_label = interpret_effect_size(r_biserial, metric="r")
median_ratio = p3_industry.median() / p3_non_industry.median() if p3_non_industry.median() > 0 else float('inf')

display(Markdown(f"""
### A.2 Sponsor effect (Phase 3)

| Sponsor | N | Median Sites |
|---------|---|--------------|
| Industry | {n1:,} | {p3_industry.median():.0f} |
| Non-industry | {n2:,} | {p3_non_industry.median():.0f} |

Mann-Whitney r = {r_biserial:.2f} ({effect_label}). Industry Phase 3 trials have {median_ratio:.1f}× higher median site count.

*Descriptive comparison only. Does not control for indication, enrollment targets, or regulatory requirements.*
"""))


### A.2 Sponsor effect (Phase 3)

| Sponsor | N | Median Sites |
|---------|---|--------------|
| Industry | 3,354 | 24 |
| Non-industry | 2,979 | 1 |

Mann-Whitney r = -0.57 (large). Industry Phase 3 trials have 24.0× higher median site count.

*Descriptive comparison only. Does not control for indication, enrollment targets, or regulatory requirements.*


---

## Appendix B: LQ Heatmap (Exploratory)

*Visual summary of §3 patterns. For reference only — do not use for decisions.*

In [16]:
# ============================================================
# B.1 LQ Heatmap
# ============================================================

top8_countries = df_country.head(8)['primary_country'].tolist()
top12_conditions = (
    df_spec
    .groupby('condition_standardized')['n_trials'].sum()
    .nlargest(12)
    .index.tolist()
)

df_heatmap_raw = df_spec[
    (df_spec['country'].isin(top8_countries)) &
    (df_spec['condition_standardized'].isin(top12_conditions))
].pivot_table(
    index='country',
    columns='condition_standardized',
    values='location_quotient',
    fill_value=1.0
)

df_n_raw = df_spec[
    (df_spec['country'].isin(top8_countries)) &
    (df_spec['condition_standardized'].isin(top12_conditions))
].pivot_table(
    index='country',
    columns='condition_standardized',
    values='n_trials',
    fill_value=0
)

df_heatmap_raw = df_heatmap_raw.reindex(top8_countries)
df_n_raw = df_n_raw.reindex(top8_countries).fillna(0)

short_names = {c: c[:18] + '...' if len(c) > 18 else c for c in df_heatmap_raw.columns}
df_heatmap = df_heatmap_raw.rename(columns=short_names)

# Calculate clipping statistics for justification
n_cells_total = df_heatmap_raw.size
n_above_2 = (df_heatmap_raw.values > 2.0).sum()
n_below_05 = (df_heatmap_raw.values < 0.5).sum()
pct_clipped = (n_above_2 + n_below_05) / n_cells_total * 100

# Display justification
display(Markdown(f"""
### B.1 Scale selection

**Why clip to [0.5, 2.0]?**

| Issue | Value |
|-------|-------|
| Cells with LQ > 2.0 | {n_above_2} ({n_above_2/n_cells_total*100:.0f}%) |
| Cells with LQ < 0.5 | {n_below_05} ({n_below_05/n_cells_total*100:.0f}%) |
| Total clipped | {pct_clipped:.0f}% |

Clipping prevents outliers from compressing the color scale. Values outside [0.5, 2.0] are labeled explicitly (">2" or "<.5") so information is not lost. The centered [0.5, 2.0] range highlights meaningful deviations from the global average (LQ = 1.0).
"""))

LOW_N_THRESHOLD = 50
text_annotations = []
low_n_count = 0
for i in range(df_heatmap_raw.shape[0]):
    row = []
    for j in range(df_heatmap_raw.shape[1]):
        raw_val = df_heatmap_raw.values[i, j]
        n_val = df_n_raw.values[i, j] if i < df_n_raw.shape[0] and j < df_n_raw.shape[1] else 0
        
        if n_val < LOW_N_THRESHOLD:
            suffix = "†"
            low_n_count += 1
        else:
            suffix = ""
        
        if raw_val > 2.0:
            row.append(f">2{suffix}")
        elif raw_val < 0.5:
            row.append(f"<.5{suffix}")
        else:
            row.append(f"{raw_val:.1f}{suffix}")
    text_annotations.append(row)

fig_heatmap = go.Figure(data=go.Heatmap(
    z=np.clip(df_heatmap.values, 0.5, 2.0),
    x=list(df_heatmap.columns),
    y=list(df_heatmap.index),
    colorscale=[
        [0, '#2166ac'],
        [0.4, '#f7f7f7'],
        [0.6, '#f7f7f7'],
        [1, '#b2182b']
    ],
    zmin=0.5,
    zmax=2.0,
    text=text_annotations,
    texttemplate='%{text}',
    textfont=dict(size=10),
    hovertemplate='<b>%{y}</b> × %{x}<br>LQ = %{z:.2f}<extra></extra>',
    colorbar=dict(title='LQ', tickvals=[0.5, 1.0, 1.5, 2.0], ticktext=['≤0.5', '1.0', '1.5', '≥2.0']),
))

fig_heatmap.update_layout(
    title=dict(
        text='<b>LQ by Country × Condition</b><br>'
             '<span style="font-size:11px;color:gray">†=n<50 | Scale clipped to [0.5, 2.0] | Exploratory only</span>',
        x=0.5, xanchor='center',
    ),
    xaxis=dict(title=None, tickangle=45, tickfont=dict(size=9)),
    yaxis=dict(title=None, tickfont=dict(size=10)),
    height=380, width=800,
    template='plotly_white',
    margin=dict(l=100, r=60, t=70, b=100),
)

fig_heatmap.show()

display(Markdown(f"*{low_n_count}/{df_heatmap_raw.size} cells have n < {LOW_N_THRESHOLD} (marked with †). See §3 for limitations.*"))


### B.1 Scale selection

**Why clip to [0.5, 2.0]?**

| Issue | Value |
|-------|-------|
| Cells with LQ > 2.0 | 4 (4%) |
| Cells with LQ < 0.5 | 20 (21%) |
| Total clipped | 25% |

Clipping prevents outliers from compressing the color scale. Values outside [0.5, 2.0] are labeled explicitly (">2" or "<.5") so information is not lost. The centered [0.5, 2.0] range highlights meaningful deviations from the global average (LQ = 1.0).


*62/96 cells have n < 50 (marked with †). See §3 for limitations.*

---

## Cleanup

In [17]:
# Close database connection
conn.close()
print("Done.")

Done.
